# MLM Phase


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import warnings
import re

from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd

!pip install wandb
!pip install Trainer
!pip install accelerate -U
!pip install transformers[torch]

warnings.filterwarnings('ignore')

import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler, Dataset, Subset

from transformers import CamembertTokenizer, CamembertTokenizerFast, CamembertForMaskedLM, CamembertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import TrainerCallback
from transformers import Trainer
from transformers import TrainingArguments

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split



In [3]:
labeled_df = pd.read_csv( "labeled_df.csv")
labeled_df = labeled_df[['label', 'id', 'text']]
labeled_df.head()

,label,id,text
0,0,02XWR02BCE,"Crise sanitaire, recrutements, succession… Les..."
1,0,03URLSTT7L,Transfert de l’énergie solaire depuis l’espace...
2,1,08XCWWYT57,Des céréales ukrainiennes vers l’Afrique : le ...
3,1,09746LR4F6,Les députés en séance plénière pour l'examen d...
4,1,09QJEPKIPQ,"L’ANSD relève une progression de 0, 8 % de l’i..."


In [4]:
def clean_text(text):
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'-{2,}', '', text)
    text = re.sub(r'\S*@\S*\s?', '', text)
    return text

labeled_df['text'] = labeled_df['text'].apply(clean_text)
labeled_df.head()

,label,id,text
0,0,02XWR02BCE,"Crise sanitaire, recrutements, succession… Les..."
1,0,03URLSTT7L,Transfert de l’énergie solaire depuis l’espace...
2,1,08XCWWYT57,Des céréales ukrainiennes vers l’Afrique : le ...
3,1,09746LR4F6,Les députés en séance plénière pour l'examen d...
4,1,09QJEPKIPQ,"L’ANSD relève une progression de 0, 8 % de l’i..."


# Phase de masquage


In [ ]:
exclusion_list = [
    '.',',',':','"','?','!','%','(',')'
    ####
    'le', 'la', 'les', 'l\'',
    ####
    'un', 'une', 'des',
    ####
    'du', 'de', "d", 'des', "'",
    ####
    'à', 'au', 'aux', 'avec', 'chez', 'contre', 'dans', 'de', 'depuis', 'derrière',
    'devant', 'durant', 'en', 'entre', 'envers', 'hors', 'jusque', 'malgré',
    'par', 'parmi', 'pendant', 'pour', 'près', 'sans', 'selon', 'sous', 'sur',
    'vers', 'via', 'voilà',
    #####
    'et', 'ou', 'mais', 'donc', 'or', 'ni', 'car', 'que',
    ####
    'je', 'tu', 'il', 'elle', 'nous', 'vous', 'ils', 'elles', 'on',
    'me', 'te', 'se', 'le', 'la', 'les', 'lui', 'leur', 'eux', 'elles',
    'ce', 'cela', 'ça', 'celui', 'celle', 'ceux', 'celles',
    'qui', 'que', 'quoi', 'dont', 'où',
    ####
    'mon', 'ton', 'son', 'notre', 'votre', 'leur', 'ma', 'ta', 'sa', 'mes', 'tes', 'ses', 'nos', 'vos', 'leurs',
    ####
    'ce', 'cet', 'cette', 'ces',
    ####
    'ne', 'pas', 'jamais', 'rien',
    #####
    'ici', 'là', 'où', 'quand', 'comment', 'pourquoi', 'combien',
    ####
    'est', 'sont', 'était', 'étaient', 'sera', 'seront', 'a', 'ont', 'avait', 'avaient', 'aura', 'auront','c’'
]

priority_vocab = ['plantation', 'élevage', 'céréale', 'légumineux', 'fruit', 'légume', 'riz', 'blé', 'maïs', 'mil', 'sorgo',
            'viande', 'sécheresse', 'pluie', 'érosion', 'désertification', 'déforestation', 'cultivée',
            'rendement', 'parasitaire', 'd\'eau', 'manque', 'niveau',
            'fourragère', 'céréalière', 'semis', 'déforestation',
            'criquet', 'agricole', 'engrai', 'bouture', 'semence',
             'pesticide', 'intrant', 'agriculture', 'rente', 'épizootie',
            'pmarché', 'prix', 'nourriture', 'soudure', 'actif',
             'misère', 'd\'achat', 'foncier',
            'source', 'faim', 'nutrition', 'calorique', 'pain', 'anémie', 'alimentaire', 'insécurité ', 'humanitaire', 'famine', 'pauvreté', 'vulnérabilité', 'crise', 'flambée', 'pèlerin',
            'insuffisante','inondation','aliment']

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import CamembertTokenizer
import torch
import numpy as np
import pandas as pd

class mlm_tokeenization(Dataset):
    def __init__(self, dataframe, tokenizer: CamembertTokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        inputs = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, truncation=True, padding='max_length', return_offsets_mapping=True)
        offset_mapping = inputs.pop('offset_mapping').squeeze()
        masked_input_ids, labels = self.apply_masking_logic(inputs['input_ids'].squeeze(), inputs['attention_mask'].squeeze())

        return {'input_ids': masked_input_ids, 'attention_mask': inputs['attention_mask'].squeeze(), 'labels': labels}

    def apply_masking_logic(self, input_ids, attention_mask):
        return mask_strategy(input_ids, attention_mask, self.tokenizer)




def mask_strategy(input_ids, attention_mask, tokenizer, masking_ratio=0.15, priority_ratio=0.3):
    seq_length = attention_mask.sum().item()
    special_tokens_mask = tokenizer.get_special_tokens_mask(input_ids.tolist(), already_has_special_tokens=True)

    priority_indices = [i for i, token in enumerate(input_ids[:seq_length]) if tokenizer.decode([token], skip_special_tokens=True).strip() in priority_vocab]
    eligible_tokens = [i for i, token in enumerate(input_ids[:seq_length]) if i not in priority_indices and special_tokens_mask[i] == 0 and tokenizer.decode([token], skip_special_tokens=True).strip() not in exclusion_list]

    total_tokens_to_mask = int(seq_length * masking_ratio)
    max_priority_tokens = int(total_tokens_to_mask * priority_ratio)

    num_priority_to_mask = min(len(priority_indices), max_priority_tokens)
    num_regular_to_mask = total_tokens_to_mask - num_priority_to_mask

    priority_masked_indices = np.random.choice(priority_indices, num_priority_to_mask, replace=False) if priority_indices else []
    regular_masked_indices = np.random.choice(eligible_tokens, num_regular_to_mask, replace=False) if eligible_tokens else []

    indices_to_mask = np.concatenate((priority_masked_indices, regular_masked_indices)).astype(int).tolist()

    labels = -100 * torch.ones_like(input_ids)

    for idx in indices_to_mask:
        labels[idx] = input_ids[idx]
        input_ids[idx] = tokenizer.mask_token_id

    return input_ids, labels





Test sur un texte


In [ ]:
tokenizer = CamembertTokenizerFast.from_pretrained('camembert-base')

test_text = "C’est tout une bonne nouvelle pour un pays où le prix du pain a déjà été multiplié par dix ces derniers mois."
test_df = pd.DataFrame({'text': [test_text]})
dataset = mlm_tokeenization(test_df, tokenizer)

item = dataset[0]

masked_text_decoded = tokenizer.decode(item['input_ids'])

print("Original text:", test_text)
print("Masked text:", masked_text_decoded)

Original text: C’est tout une bonne nouvelle pour un pays où le prix du pain a déjà été multiplié par dix ces derniers mois.
Masked text: <s><mask>’est tout une bonne nouvelle pour un pays où le prix du<mask> a déjà été multiplié par<mask> ces derniers<mask>.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
from sklearn.model_selection import train_test_split


train_df, temp_df = train_test_split(labeled_df, test_size=0.5, random_state=42)

val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)



In [ ]:
train_dataset = mlm_tokeenization(train_df['text'], tokenizer)
val_dataset = mlm_tokeenization(val_df['text'], tokenizer)
test_dataset = mlm_tokeenization(test_df['text'], tokenizer)

In [ ]:
import accelerate
print(accelerate.__version__)

0.28.0


In [ ]:
from transformers import EvalPrediction

model = CamembertForMaskedLM.from_pretrained('camembert-base')

def compute_metrics(p: EvalPrediction):
    predictions = p.predictions
    labels = p.label_ids
    pred_flat = np.argmax(predictions, axis=2).flatten()
    labels_flat = labels.flatten()
    mask = labels_flat != -100
    correct = (pred_flat[mask] == labels_flat[mask]).sum()
    total = mask.sum()

    return {"accuracy": correct / total if total > 0 else 0}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=CamemBERTDataset(train_df, tokenizer),
    eval_dataset=CamemBERTDataset(val_df, tokenizer),
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.926495,0.605073
2,No log,1.834873,0.617663
3,No log,1.759344,0.634512


TrainOutput(global_step=129, training_loss=1.9730045998743337, metrics={'train_runtime': 46.4965, 'train_samples_per_second': 10.904, 'train_steps_per_second': 2.774, 'total_flos': 133447149868032.0, 'train_loss': 1.9730045998743337, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_directory = "/content/drive/My Drive/MyModel"
trainer.save_model(save_directory)
#tokenizer.save_pretrained(save_directory)

In [ ]:
from transformers import CamembertForMaskedLM, CamembertTokenizer

model = CamembertForMaskedLM.from_pretrained(save_directory)
tokenizer = CamembertTokenizer.from_pretrained(save_directory)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model2 = CamembertForMaskedLM.from_pretrained('camembert-base')
tokenizer2 = CamembertTokenizerFast.from_pretrained('camembert-base')

def predict_masked_text(text):
    input_ids = tokenizer2.encode(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    masked_indices = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()
    result = []

    for i, masked_index in enumerate(masked_indices, start=1):
        k = 5
        topk = torch.topk(predictions[0, masked_index], k)
        topk_tokens = [tokenizer.decode([idx]).strip() for idx in topk.indices.tolist()]
        topk_probabilities = torch.softmax(topk.values, dim=0).tolist()  # Convert logits to probabilities

        prediction_strings = [f"{token}: {prob:.2%}" for token, prob in zip(topk_tokens, topk_probabilities)]
        result.append(f"Predicted word {i}. Top {k}:\n" + "\n".join(prediction_strings))

    return "\n\n".join(result)

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

In [ ]:
# Create Gradio interface
!pip install gradio
import gradio as gr

iface = gr.Interface(
    fn=predict_masked_text,
    inputs="text",
    outputs="text",
    examples=[
        ["En Afrique, la <mask> <mask> reste un défi persistant, marqué par des <mask> considérables. Malgré quelques progrès, de nombreuses communautés continuent de <mask> pour accéder à une alimentation adéquate et nutritive. Les infrastructures agricoles sont souvent insuffisantes, entraînant des pertes alimentaires importantes. De plus, les politiques et les investissements sont parfois inéquitables, laissant de côté les populations les plus marginalisées. Face à ces obstacles, il est crucial de redoubler d'efforts pour garantir un avenir où chaque individu en Afrique puisse vivre sans craindre la <mask> ni la <mask>."]
    ],
    title="MLM Predictions for Masked Text",
    description="This model predicts the top replacements for masked words in a sentence. Enter a sentence with <mask> tokens to see predictions."
)

iface.launch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.7 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl si

# Binary classification phase

In [ ]:
#### train val test

In [2]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import re
from google.colab import drive
drive.mount('/content/drive')

!pip install Trainer
!pip install accelerate -U
!pip install transformers[torch]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
labeled_df = pd.read_csv("labeled_df.csv")
labeled_df = labeled_df[['label', 'id', 'text']]
special_case = labeled_df[(labeled_df['id'] == 'NDBP5YOL02') & (labeled_df['label'] == 1)]
if not special_case.empty:

    labeled_df = labeled_df.drop(labeled_df[(labeled_df['id'] == 'NDBP5YOL02')].index)

    labeled_df = pd.concat([labeled_df, special_case], ignore_index=True)


labeled_df = labeled_df.drop_duplicates(subset=['id'], keep='first')
print(labeled_df[(labeled_df['id'] == 'NDBP5YOL02') ])


     label          id                                               text
337      1  NDBP5YOL02  Gestion des cantines scolaires au Bénin : Une ...


In [4]:
df = pd.read_csv("merged_predictions_with_relevance.csv")

num_rows = len(df)
print(f"Number of rows: {num_rows}")

df.head()

Number of rows: 9436


,label,id,text,predicted_label,confidence,CLS_Relevance Yes/No
0,NaN,DMAMGYZL53,La proposition du PRD sur la règle de 10%\n\nL...,0,0.857464,No (m)
1,NaN,TTRAZK9XGJ,[ VIDEO ] ------------- (ex-Destiny’s Child) m...,0,0.849603,No (m)
2,NaN,T2YIHR1I2U,"Égypte : La FSBB exprime sa "" préoccupation "" ...",0,0.853480,No (m)
3,NaN,7JS87XQ62T,"(Dossier) Cherté du supercarburant, hausse des...",1,0.705428,Yes (m)
4,NaN,A6CVE0463M,Laser du lundi : Quand ---------- se meut entr...,0,0.855187,Yes (m)


In [5]:
unique_ids_in_labeled_df = labeled_df['id'].unique()

df = df[~df['id'].isin(unique_ids_in_labeled_df)]

In [6]:
def clean_text(text):
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'-{2,}', '', text)
    text = re.sub(r'\S*@\S*\s?', '', text)
    return text

In [7]:
df['text'] = df['text'].apply(clean_text)
labeled_df['text'] = labeled_df['text'].apply(clean_text)

display(labeled_df.head())
display(df.head())

label_counts = labeled_df['label'].value_counts()
print("Counts of each label:")
print(label_counts)

,label,id,text
0,0,02XWR02BCE,"Crise sanitaire, recrutements, succession… Les..."
1,0,03URLSTT7L,Transfert de l’énergie solaire depuis l’espace...
2,1,08XCWWYT57,Des céréales ukrainiennes vers l’Afrique : le ...
3,1,09746LR4F6,Les députés en séance plénière pour l'examen d...
4,1,09QJEPKIPQ,"L’ANSD relève une progression de 0, 8 % de l’i..."


,label,id,text,predicted_label,confidence,CLS_Relevance Yes/No
0,NaN,DMAMGYZL53,La proposition du PRD sur la règle de 10%\n\nL...,0,0.857464,No (m)
1,NaN,TTRAZK9XGJ,[ VIDEO ] (ex-Destiny’s Child) montre ses sei...,0,0.849603,No (m)
2,NaN,T2YIHR1I2U,"Égypte : La FSBB exprime sa "" préoccupation "" ...",0,0.853480,No (m)
4,NaN,A6CVE0463M,Laser du lundi : Quand se meut entre étau et ...,0,0.855187,Yes (m)
5,NaN,198HBJZBRZ,Covid en Chine : le bilan s'alourdit à Shangha...,0,0.867945,No (m)


Counts of each label:
label
1    170
0    164
Name: count, dtype: int64


#Conversion de certains textes mal étiquetés

In [ ]:
search_texts = [
    "Foire de l’énergie solaire: les opportunités de formation et les solutions innovantes dévoilées à Cotonou",
    "Bénin : environ 210 millions de dollars investis en 03 ans par les Nations Unies",
    "Autoroute de Saint-Louis : les travaux démarrent en juin avec l’arrivée des premiers financements",
    "Contrôle des huiles alimentaires vendues sur le marché burkinabè : propose de renforcer la veille",
    "IFC et des partenaires s’allient pour soutenir le développement du secteur (...)",
    "au sujet des écosystèmes de mangroves : ",
    "édition du Forum international de Dakar sur la paix et la sécurité en Afrique"
    "Famine, inflation… La guerre en Ukraine nous ferait regretter la"
]

def invert_label(label):
    return 1 if label == 0 else 0


for search_text in search_texts:t
    idx = labeled_df[labeled_df['text'].str.contains(search_text, na=False, case=False)].index
    if len(idx) > 0:
        for i in idx:
            old_label = labeled_df.loc[i, 'label']
            labeled_df.loc[i, 'label'] = invert_label(old_label)
            print(f"Modified text:\n{labeled_df.loc[i, 'text']}\nOld label: {old_label}, New label: {labeled_df.loc[i, 'label']}\n")
    else:
        print(f"No match found for text: {search_text}")


# Tokenization: 256 premiers, 256 derniers tokens

In [9]:
class tokenization(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer.encode_plus(text, add_special_tokens=False, max_length=self.max_length, truncation=True)

        start_tokens = encoding['input_ids'][:255]  # 256 - 1
        end_tokens = encoding['input_ids'][-(self.max_length - len(start_tokens) - 3):]  # -3

        input_ids = [self.tokenizer.cls_token_id] + start_tokens + [self.tokenizer.sep_token_id] + end_tokens + [self.tokenizer.sep_token_id]
        attention_mask = [1] * len(input_ids)

        padding_length = self.max_length - len(input_ids)
        input_ids += [self.tokenizer.pad_token_id] * padding_length
        attention_mask += [0] * padding_length

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }


In [23]:
labeled_df = shuffle(labeled_df, random_state=5)

train_val_df, test_df = train_test_split(labeled_df, test_size=0.15, stratify=labeled_df['label'])
train_df, val_df = train_test_split(train_val_df, test_size=0.10, stratify=train_val_df['label'])

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [24]:
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
train_dataset = tokenization(train_df['text'], train_df['label'], tokenizer)
val_dataset = tokenization(val_df['text'], val_df['label'], tokenizer)
test_dataset = tokenization(test_df['text'], test_df['label'], tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
#model = CamembertForSequenceClassification.from_pretrained("/content/drive/My Drive/MyModel", num_labels=2)

model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=2)

import shutil
import os

checkpoint_dir = './results'
if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
os.makedirs(checkpoint_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=3e-5,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    warmup_steps=50,
    weight_decay=0.02,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",  # on sauvegarde le meuilleur en terme d'accuracy
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

#################
trainer.train()
################

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

conf_matrix = confusion_matrix(test_dataset.labels, pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
report = classification_report(test_dataset.labels, pred_labels, output_dict=True)
print(classification_report(test_dataset.labels, pred_labels))

precision, recall, _, _ = precision_recall_fscore_support(test_dataset.labels, pred_labels)
plt.figure(figsize=(10, 5))
barWidth = 0.3
r1 = np.arange(len(precision))
r2 = [x + barWidth for x in r1]

plt.bar(r1, precision, color='b', width=barWidth, label='Precision')
plt.bar(r2, recall, color='r', width=barWidth, label='Recall')

plt.xlabel('Class')
plt.xticks([r + barWidth for r in range(len(precision))], ['Class 0', 'Class 1'])
plt.ylabel('Score')
plt.title('Precision and Recall by Class')
plt.legend()
plt.show()

In [ ]:
actual_labels = test_dataset.labels

wrong_indices = [i for i,   (pred, actual) in  enumerate(zip(pred_labels, actual_labels)) if pred != actual]


wrongly_classified_texts = [test_dataset.texts[i]   for i in wrong_indices]
wrong_predictions =pred_labels[wrong_indices]
actual_wrong_labels =[actual_labels[i]  for i in wrong_indices]

for i in range(min(18, len(wrongly_classified_texts))):
    print(f"Text :  {wrongly_classified_texts[i]}")
    print(f"Predicted Label:   {wrong_predictions[i]}, Actual Label: {actual_wrong_labels[i]}\n###############")


In [39]:
save_directory = "/content/drive/My Drive/MyModel_0405"

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/My Drive/MyModel_0405/tokenizer_config.json',
 '/content/drive/My Drive/MyModel_0405/special_tokens_map.json',
 '/content/drive/My Drive/MyModel_0405/sentencepiece.bpe.model',
 '/content/drive/My Drive/MyModel_0405/added_tokens.json')

In [ ]:
from transformers import CamembertForSequenceClassification, CamembertTokenizer
#save_directory = "/content/drive/My Drive/MyModelPretrainedTest2"
#model = CamembertForSequenceClassification.from_pretrained(save_directory)

#tokenizer = CamembertTokenizer.from_pretrained(save_directory)


trainer = Trainer(model=model)
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_dataset.labels, pred_labels)
print(f"Accuracy: {accuracy}")

In [ ]:
actual_labels = test_dataset.labels

wrong_indices = [i for i,   (pred, actual)  in enumerate(zip(pred_labels, actual_labels)) if pred != actual]

wrongly_classified_texts =[test_dataset.texts[i] for i in wrong_indices]
wrong_predictions = pred_labels[wrong_indices]
actual_wrong_labels =[actual_labels[i] for i in wrong_indices]

for i in range(min(7, len(wrongly_classified_texts))):
    print(f"Text: {wrongly_classified_texts[i]}")
    print(f"Predicted Label:   {wrong_predictions[i]}, Actual Label: {actual_wrong_labels[i]}\n###############")


In [32]:
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

class InferenceDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

texts = df['text'].tolist()

encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')

inference_dataset = InferenceDataset(encodings)

dataloader = DataLoader(inference_dataset, batch_size=64)

model.eval()


model.to(model.device)


predicted_labels = []
confidence_scores = []



progress_bar = tqdm(dataloader, desc="Predicting")
with torch.no_grad():

    for batch in progress_bar:
        outputs=model(**{k: v.to(model.device)  for k, v in batch.items()} )
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        top_probs, top_lbls = torch.max(probs, dim=1)

        predicted_labels.extend(top_lbls.cpu().numpy())
        confidence_scores.extend(top_probs.cpu().numpy())



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Predicting:   0%|          | 0/148 [00:00<?, ?it/s]

In [ ]:
from collections import Counter

label_counts = Counter(predicted_labels)

print("Number of labels 0:", label_counts[0])
print("Number of labels 1:", label_counts[1])

In [35]:
df['predicted_label_noMLM'] =  predicted_labels
df['confidence_noMLM'] = confidence_scores

In [37]:
mismatches = df[df['predicted_label'] != df['predicted_label_noMLM']]

for _, row in mismatches.iterrows():
    print(f"ID: {row['id']}, Previous Prediction: {row['predicted_label']}, New Prediction: {row['predicted_label_noMLM']}, Confidence: {row['confidence_noMLM']}")

ID: RJ5H4F3A8T, Previous Prediction: 0, New Prediction: 1, Confidence: 0.5652191042900085
ID: MAOS6IZWSH, Previous Prediction: 0, New Prediction: 1, Confidence: 0.6853719353675842
ID: LK3SZYWQ7M, Previous Prediction: 0, New Prediction: 1, Confidence: 0.7362664937973022
ID: 6DWT2E7TIQ, Previous Prediction: 0, New Prediction: 1, Confidence: 0.8722246289253235
ID: FTSY5YJN8Z, Previous Prediction: 0, New Prediction: 1, Confidence: 0.6297280788421631
ID: NH0BKSSL2W, Previous Prediction: 0, New Prediction: 1, Confidence: 0.7426002621650696
ID: W0UCFG80AZ, Previous Prediction: 0, New Prediction: 1, Confidence: 0.8742474317550659
ID: MOL7M8WKHA, Previous Prediction: 0, New Prediction: 1, Confidence: 0.7588765621185303
ID: MK0JQR3D9P, Previous Prediction: 0, New Prediction: 1, Confidence: 0.8847898244857788
ID: VZCR9FFMYB, Previous Prediction: 0, New Prediction: 1, Confidence: 0.578697681427002
ID: 6AV6H9BJ0K, Previous Prediction: 0, New Prediction: 1, Confidence: 0.6555342078208923
ID: FPIOQTU

In [38]:
df.to_csv("/content/drive/My Drive/updated_predictions_nomlm.csv", index=False)